In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import open3d as o3d
from torch.utils.data import DataLoader

# Learning3D kütüphanesinden PPFNet ve ChamferLoss fonksiyonu
from learning3d.models import PPFNet
from learning3d.losses import ChamferDistanceLoss
from learning3d.data_utils import ModelNet40Data  # ModelNet40 veri kümesi

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
def compute_normals_single(points_single):
    """
    Tek bir nokta bulutu (N, 3) için Open3D kullanarak yüzey normallerini hesaplar.
    """
    pcd = o3d.geometry.PointCloud()
    # Noktaların numpy array'inin tipini float64'e dönüştürüyoruz
    pcd.points = o3d.utility.Vector3dVector(points_single.astype(np.float64))
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    normals = np.asarray(pcd.normals)
    return normals

In [4]:
def augment(points):
    noise = torch.randn_like(points) * 0.02  # Jittering
    angle = torch.rand(1) * 2 * torch.pi  # Random rotation
    rotation_matrix = torch.tensor([
        [torch.cos(angle), -torch.sin(angle), 0],
        [torch.sin(angle), torch.cos(angle), 0],
        [0, 0, 1]
    ], dtype=torch.float32).to(points.device)
    
    points = torch.matmul(points, rotation_matrix)  # Döndür
    points = points + noise  # Gürültü ekle
    return points

In [5]:
train_dataset = ModelNet40Data(train=True, num_points=1024)  
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

In [6]:
class PointNetPPDecoder(nn.Module):
    def __init__(self, embedding_dim=32, num_points=1024):
        super(PointNetPPDecoder, self).__init__()
        self.num_points = num_points

        # Katmanlar: Boyutları kademeli artır
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, num_points * 3)
        )
    
    def forward(self, embedding):
        x = self.fc_layers(embedding)  # (B, num_points * 3)
        recon = x.view(-1, self.num_points, 3)  # (B, num_points, 3)
        return recon

In [7]:
class PPFNetAutoEncoder(nn.Module):
    def __init__(self, embedding_dim=32, num_points=1024):
        super(PPFNetAutoEncoder, self).__init__()
        # PPFNet encoder: burada kullanılacak parametreleri isteğinize göre ayarlayın
        self.encoder = PPFNet(features=['ppf', 'dxyz', 'xyz'], emb_dims=embedding_dim, radius=0.3, num_neighbors=64)
        # Basit decoder: global embedding'i alıp nokta koordinatlarını üretir.
        self.decoder = PointNetPPDecoder(embedding_dim=32, num_points=1024)
    
        self.num_points = num_points
    
    def forward(self, points, normals):
        # Encoder: PPFNet, points ve normals alır ve per nokta özelliklerini üretir → (B, embedding_dim, N)
        features = self.encoder(points, normals)
        # Global özellik: noktalar üzerinde max pooling → (B, embedding_dim)
        global_embedding = torch.max(features, dim=1)[0]
        # Decoder: global embedding'i kullanarak yeniden nokta bulutu üret
        recon = self.decoder(global_embedding)  # (B, num_points*3)
        recon = recon.view(-1, self.num_points, 3)  # (B, num_points, 3)
        return global_embedding, recon

# Modeli oluştur ve cihaza taşı
model = PPFNetAutoEncoder(embedding_dim=32, num_points=1024).to(device)

In [8]:
criterion = ChamferDistanceLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)

In [9]:
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for i, data in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        # Veri kümesinden noktaları al; etiketler bu autoencoder için kullanılmayacak.
        points, _ = data  # Sadece nokta bulutu verisi
        points = points - points.mean(dim=1, keepdim=True)  # Merkezi sıfıra getir
        points = points / torch.max(torch.norm(points, dim=-1, keepdim=True))  # Birime normalize et
        points = augment(points)
        
        points = points.to(device, dtype=torch.float32)
        
        # Batch içindeki her örnek için normalleri hesaplayın.
        points_np = points.cpu().numpy()  # Şekil: (B, N, 3)
        normals_list = []
        for pc in points_np:
            normals_pc = compute_normals_single(pc)  # Her pc: (N, 3)
            normals_list.append(normals_pc)
        normals_np = np.stack(normals_list, axis=0)  # (B, N, 3)
        normals = torch.tensor(normals_np, dtype=torch.float32).to(device)
        
        # İleri geçiş
        embedding, recon_points = model(points, normals)
        
        # Kayıp hesaplama: Yeniden oluşturulan nokta bulutu ile orijinal arasındaki chamfer distance
        loss = criterion(recon_points, points) + 0.01 * torch.norm(embedding, p=2)
        epoch_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

print("Eğitim tamamlandı!")

Epoch 1/30: 100%|███████████████████████████| 1230/1230 [01:01<00:00, 19.86it/s]


Epoch [1/30], Loss: 0.1027, LR: 0.000001


Epoch 2/30: 100%|███████████████████████████| 1230/1230 [01:01<00:00, 19.90it/s]


Epoch [2/30], Loss: 0.0877, LR: 0.001000


Epoch 3/30: 100%|███████████████████████████| 1230/1230 [01:01<00:00, 20.09it/s]


...

Epoch [28/30], Loss: 0.0696, LR: 0.001000


Epoch 29/30: 100%|██████████████████████████| 1230/1230 [01:01<00:00, 20.08it/s]


Epoch [29/30], Loss: 0.0696, LR: 0.000001


Epoch 30/30: 100%|██████████████████████████| 1230/1230 [01:00<00:00, 20.18it/s]

Epoch [30/30], Loss: 0.0693, LR: 0.001000
Eğitim tamamlandı!


In [10]:
# Autoencoder modelini kaydet
torch.save(model.decoder.state_dict(), "ppfnet32_decoder.pth")

# Sadece Encoder kısmını kaydet
torch.save(model.encoder.state_dict(), "ppfnet32_encoder.pth")